# Notebook: Create Examples for Prompts

## Packages

In [34]:
from helper_synthesis import get_examples_as_text
from itertools import cycle, islice
import numpy as np
import tiktoken
import random
import json

## Parameters

In [35]:
SEED = 42
random.seed(SEED)
N_RETRY_SETS = 25

In [36]:
# Setup Classes/Polarities for Synthesis
CLASSES  = ["GENERAL-IMPRESSION", "FOOD", "SERVICE", "AMBIENCE", "PRICE"]
POLARITIES = ["POSITIVE", "NEUTRAL", "NEGATIVE"]
COMBINATIONS = [(aspect, polarity) for polarity in POLARITIES for aspect in CLASSES]

## Code

### Code to Count Prompt Size

In [37]:
with open('../prompt_template.txt', 'r') as file:
    PROMPT_TEMPLATE = file.read()

In [38]:
with open(f"../07 train models/real/split_4.json", 'r', encoding='utf-8') as json_file:
    dataset = json.load(json_file)

In [39]:
label = [('GENERAL-IMPRESSION', 'POSITIVE')]

In [40]:
examples = [example["id"] for example in dataset[:40]]

In [41]:
random.shuffle(dataset)

In [42]:
def get_n_examples_for_class(n_examples=5, aspect_category="FOOD", examples=[]):
    found_examples = []
    i = 0
    while len(found_examples) < n_examples:
        if aspect_category in [tag["label"] for tag in dataset[i]["tags"]] and dataset[i]["id"] not in examples:
            found_examples.append(dataset[i])
        i += 1
    return examples + [example["id"] for example in found_examples]


In [43]:
examples = []
for ac in CLASSES:
    n_examples = 5
    examples = get_n_examples_for_class(n_examples=n_examples, aspect_category=ac, examples=examples)
random.shuffle(examples)
        

In [44]:
few_shot_examples = [entry for entry in dataset if entry['id'] in examples]

examples_text = get_examples_as_text(few_shot_examples)
prompt_footer = f'\nLabel:{str(label)}\nPrediction:'
prompt = PROMPT_TEMPLATE + examples_text + prompt_footer
print(prompt)

Erzeuge genau einen Satz einer deutschsprachigen Restaurant-Bewertung, der für das Training eines Modells für die Aspekt-basierte Sentiment Analyse verwendet werden kann.
Gegeben ist ein Label in Form eines Arrays, wobei ein oder mehrere Tupel (Aspekt-Kategorie, Sentiment-Polarität) gegeben sind.
Für ein Label wird eine Prediction erzeugt, wobei die Prediction ausschließlich die in dem Label definierten Kombinationen aus Aspekt-Kategorie und Aspekt-Polarität adressiert.

Folgende Aspekt-Kategorien werden betrachtet: 

* "FOOD" - Aspekte in Bezug auf das Essen im allgemeinen oder bestimmte Speisen und Getränke
* "SERVICE" - Aspekte in Bezug auf den Service im allgemeinen oder Einstellung und Professionalität des Personals, die Wartezeiten oder Service-Dienstleistungen wie Speisenmitnahme
* "PRICE" - Aspekte in Bezug auf den Preis im allgemeinen oder Speisen, Getränke oder andere Leistungen des Restaurants, deren Preis bewertet wird.
* "AMBIENCE" - Aspekte in Bezug auf das Ambiente und A